In [6]:
import matplotlib.pyplot as plt
import numpy as np
#card library
import spcm
from spcm import units # spcm uses the pint library for unit handling (units is a UnitRegistry object)
import datetime
import threading
import distinctipy as dc

In [11]:
global EXIT_FLAG
EXIT_FLAG=False

def DAQCard(trigger_level,#=2200, # mV
            sampling_rate,#=0.1, #MHz
            amplitude,#=5000, # mV
            channels_to_use,#[0,3,5],
            trigger_channel,#=0,
            post_trig_duarat,#=1, # ms
            pretrig_duaration,#=0.1,#ms
            show_signals,#=False,
            show_signals_ax,#=None,
            info_print,#=True,
            #processing
            snip_size,#500
            preprocessing,
            #the ML model
            s_model,
            colorscheme,#this is to show
            fig,
            fig_ax,
            ):
    
    #import SHelpers as shlp
    
#    EXIT_FLAG=exit_flag#False
    TRIGGER_LEVEL=trigger_level#2200 # mV
    SAMPLING_RATE=sampling_rate#0.1 #MHz
    AMPLITUDE=amplitude#5000 # mV
    CHANNELS_TO_USE=channels_to_use#[0,3,5]
    TRIGGER_CHANNEL=trigger_channel#0
    POSTTRIG_DURATION=post_trig_duarat #1 # ms
    PRETRIG_DURATION=pretrig_duaration#0.1#ms
    SHOW_SIGNALS=show_signals#False
    SHOW_SIGNALS_AX=show_signals_ax#None
    DATA_PRINT=info_print#True
    #https://spectruminstrumentation.github.io/spcm/spcm.html#Trigger
    #def SPECTRUM_DAQ():
    signals_counter=0
    signals=[]
    for i in range(0,8):
        signals.append([])        
    card:spcm.Card
    #prepare the figure
    #with spcm.Card('/dev/spcm0') as card:                         # if you want to open a specific card
    # with spcm.Card('TCPIP::192.168.1.10::inst0::INSTR') as card:  # if you want to open a remote card
    # with spcm.Card(serial_number=12345) as card:                  # if you want to open a card by its serial number
      
    with spcm.Card(card_type=spcm.SPCM_TYPE_AI) as card:            # if you want to open the first card of a specific type
        ##threading termination - https://stackoverflow.com/questions/18018033/how-to-stop-a-looping-thread-in-python
        t = threading.current_thread()
        while(getattr(t, "do_run", True)):   
            if(EXIT_FLAG==True):
                print("Data acquisition is terminating...")
                return
            
            # do a simple standard setup
            card.card_mode(spcm.SPC_REC_STD_SINGLE)     # single trigger standard mode
            card.timeout(5 * units.s)                     # timeout 5 s
        
            trigger = spcm.Trigger(card)
            trigger.or_mask(spcm.SPC_TMASK_NONE)       # trigger set to none #software
            trigger.and_mask(spcm.SPC_TMASK_NONE)      # no AND mask
        
            clock = spcm.Clock(card)
            clock.mode(spcm.SPC_CM_INTPLL)            # clock mode internal PLL
            clock.sample_rate(SAMPLING_RATE * units.MHz, return_unit=units.MHz)
        
            # setup the channels
            #channel0
            channel0, = spcm.Channels(card, card_enable=spcm.CHANNEL0) # enable channel 0
            channel0.amp(AMPLITUDE * units.mV)
            channel0.offset(0 * units.mV)
            channel0.termination(0)
            #channel1
            channel1, = spcm.Channels(card, card_enable=spcm.CHANNEL1) # enable channel 1
            channel1.amp(AMPLITUDE * units.mV)
            channel1.offset(0 * units.mV)
            channel1.termination(0)
            #channel2
            channel2, = spcm.Channels(card, card_enable=spcm.CHANNEL2) # enable channel 1
            channel2.amp(AMPLITUDE * units.mV)
            channel2.offset(0 * units.mV)
            channel2.termination(0)
            #channel3
            channel3, = spcm.Channels(card, card_enable=spcm.CHANNEL3) # enable channel 1
            channel3.amp(AMPLITUDE * units.mV)
            channel3.offset(0 * units.mV)
            channel3.termination(0)
            #channel4
            channel4, = spcm.Channels(card, card_enable=spcm.CHANNEL4) # enable channel 1
            channel4.amp(AMPLITUDE * units.mV)
            channel4.offset(0 * units.mV)
            channel4.termination(0)
            #channel5
            channel5, = spcm.Channels(card, card_enable=spcm.CHANNEL5) # enable channel 1
            channel5.amp(AMPLITUDE * units.mV)
            channel5.offset(0 * units.mV)
            channel5.termination(0)
            #channel6
            channel6, = spcm.Channels(card, card_enable=spcm.CHANNEL6) # enable channel 1
            channel6.amp(AMPLITUDE * units.mV)
            channel6.offset(0 * units.mV)
            channel6.termination(0)
            #channel7
            channel7, = spcm.Channels(card, card_enable=spcm.CHANNEL7) # enable channel 1
            channel7.amp(AMPLITUDE * units.mV)
            channel7.offset(0 * units.mV)
            channel7.termination(0)

            if(EXIT_FLAG==True):
                print("Data acquisition is terminating...")
                return
            
            # Channel triggering
            if(TRIGGER_CHANNEL==0):                
                trigger.and_mask(spcm.SPC_TMASK_NONE)
                trigger.or_mask(spcm.SPC_TMASK_EXT0)
                trigger.ext0_mode(spcm.SPC_TM_POS)
                trigger.ext0_level0(float(TRIGGER_LEVEL) * units.mV)
                trigger.ext0_coupling(spcm.COUPLING_DC)
                trigger.termination(termination=0)                
                #print("Trigger channel 0")
            if(TRIGGER_CHANNEL==1):                
                trigger.and_mask(spcm.SPC_TMASK_NONE)
                trigger.or_mask(spcm.SPC_TMASK_EXT1)
                trigger.ext1_mode(spcm.SPC_TM_POS)
                trigger.ext1_coupling(spcm.COUPLING_DC)
                trigger.termination(termination=0)
                trigger.ext1_level0(float(TRIGGER_LEVEL) * units.mV)
                print("Trigger channel 1")
            if(TRIGGER_CHANNEL==2):                
                trigger.and_mask(spcm.SPC_TMASK_NONE)
                trigger.or_mask(spcm.SPC_TMASK_EXT2)
                trigger.ext2_mode(spcm.SPC_TM_POS)
                trigger.ext2_coupling(spcm.COUPLING_DC)
                trigger.termination(termination=0)
                trigger.ext2_level0(float(TRIGGER_LEVEL) * units.mV)
                
            if(TRIGGER_CHANNEL==3):                
                trigger.and_mask(spcm.SPC_TMASK_NONE)
                trigger.or_mask(spcm.SPC_TMASK_EXT3)
                trigger.ext3_mode(spcm.SPC_TM_POS)
                trigger.ext3_coupling(spcm.COUPLING_DC)
                trigger.termination(termination=0)
                trigger.ext3_level0(float(TRIGGER_LEVEL) * units.mV)                
            if(TRIGGER_CHANNEL==4):                
                trigger.and_mask(spcm.SPC_TMASK_NONE)
                trigger.or_mask(spcm.SPC_TMASK_EXT4)
                trigger.ext4_mode(spcm.SPC_TM_POS)
                trigger.ext4_coupling(spcm.COUPLING_DC)
                trigger.termination(termination=0)
                trigger.ext4_level0(float(TRIGGER_LEVEL) * units.mV)
                
            if(TRIGGER_CHANNEL==5):                
                trigger.and_mask(spcm.SPC_TMASK_NONE)
                trigger.or_mask(spcm.SPC_TMASK_EXT5)
                trigger.ext5_mode(spcm.SPC_TM_POS)
                trigger.ext5_coupling(spcm.COUPLING_DC)
                trigger.termination(termination=0)
                trigger.ext5_level0(float(TRIGGER_LEVEL) * units.mV)
                
            if(TRIGGER_CHANNEL==6):                
                trigger.and_mask(spcm.SPC_TMASK_NONE)
                trigger.or_mask(spcm.SPC_TMASK_EXT6)
                trigger.ext6_mode(spcm.SPC_TM_POS)
                trigger.ext6_coupling(spcm.COUPLING_DC)
                trigger.termination(termination=0)
                trigger.ext6_level0(float(TRIGGER_LEVEL) * units.mV)
                
            if(TRIGGER_CHANNEL==7):                
                trigger.and_mask(spcm.SPC_TMASK_NONE)
                trigger.or_mask(spcm.SPC_TMASK_EXT7)
                trigger.ext7_mode(spcm.SPC_TM_POS)
                trigger.ext7_coupling(spcm.COUPLING_DC)
                trigger.termination(termination=0)
                trigger.ext7_level0(float(TRIGGER_LEVEL) * units.mV)
                
            # define the data buffer
            data_transfer = spcm.DataTransfer(card)
            data_transfer.duration((PRETRIG_DURATION+POSTTRIG_DURATION)*units.ms, post_trigger_duration=POSTTRIG_DURATION*units.ms)
        
            if(True):#while(True)

                if(EXIT_FLAG==True):
                    print("Data acquisition is terminating...")
                    return
                
                # start card and wait until recording is finished
                card.start(spcm.M2CMD_CARD_ENABLETRIGGER, spcm.M2CMD_CARD_WAITREADY)
                #print("Finished acquiring...")
                
                # Start DMA transfer and wait until the data is transferred
                data_transfer.start_buffer_transfer(spcm.M2CMD_DATA_STARTDMA, spcm.M2CMD_DATA_WAITDMA)
            
                signals[0]=channel0.convert_data(data_transfer.buffer[channel0, :], units.V)
                signals[1]=channel1.convert_data(data_transfer.buffer[channel1, :], units.V)
                signals[2]=channel2.convert_data(data_transfer.buffer[channel2, :], units.V)
                signals[3]=channel3.convert_data(data_transfer.buffer[channel3, :], units.V)
                signals[4]=channel4.convert_data(data_transfer.buffer[channel4, :], units.V)
                signals[5]=channel5.convert_data(data_transfer.buffer[channel5, :], units.V)
                signals[6]=channel6.convert_data(data_transfer.buffer[channel6, :], units.V)
                signals[7]=channel7.convert_data(data_transfer.buffer[channel7, :], units.V)

                #**********************************************************************
                #*********************************processing***************************
                """
                plate=shlp.SPlate()
                plate.raw_signals=signals
                plate.get_segments(ref_chan_name=trigger_channel)
                sgn_len = len(plate.sigments_sign)
                all_labels=[]
                all_segm_sign=[]
                for seg_i in range(0,sgn_len):
                        #take segment
                        segm_signal=plate.sigments_sign[seg_i]
                        snippets=data_proc.SplitEntireSignalIntoSnippets(signal=segm_signal,
                                                                         channs_indx=CHANNELS_TO_USE,
                                                                         torch_tensor=False,
                                                                         snip_size = snip_size,
                                                                         preproc_type = preprocessing
                                                                         )
                        start_t=time.time()
                        #labels=s_model.np_predict(np.asarray(snippets))
                        end_t=time.time()
                        #all_labels.append(labels) 
                        #all_segm_sign.append(segm_signal)
                    
                #show everything in the chart   
                if(fig is not None) and (fig_ax is not None):
                    try:               
                        viz_start_t=time.time()
                        #shlp.ShowResultsInFigure_AllSegmentsInRow(all_segm_sign,all_labels,CHANNELS_TO_USE,snip_size,colorscheme,fig,fig_ax,None)
                        viz_end_t=time.time()
                    except:
                        print("Figure is anavailable.")
                """
                #***************************end***processing***************************
                #**********************************************************************
                
                if(DATA_PRINT==True):                    
                    now = datetime.datetime.now()
                    signals_counter+=1
                    trig_time_st=int((PRETRIG_DURATION/1000)*SAMPLING_RATE*1000000)
                    print("")
                    print("******************************************")
                    print("Data received: "+str(now))
                    print("Data length/chan.: "+str(len(signals[0])))
                    print("Chans.num.: "+str(len(signals)))
                    print("Frame num.:"+str(signals_counter))
                    print("Trig. chan.: "+str(TRIGGER_CHANNEL))
                    print("Trig.timestamp: "+str(trig_time_st/1000)+" ms")
                    print("Trig. val.:"+str(signals[TRIGGER_CHANNEL][trig_time_st]))
                    print("******************************************")
                    print("")
                
                # Plot the acquired data
                if(SHOW_SIGNALS==True):            
                    time_data_s = data_transfer.time_data()
                    #fig, ax = plt.subplots()
                    unit_data_V = channel0.convert_data(data_transfer.buffer[channel0, :], units.V)
                    print(channel0)
                    print("\tMinimum: {:.3~P}".format(np.min(unit_data_V)))
                    print("\tMaximum: {:.3~P}".format(np.max(unit_data_V)))                    
                    SHOW_SIGNALS_AX.clear()
                    SHOW_SIGNALS_AX.plot(time_data_s, signals[0], label=f"{channel0}")
                    SHOW_SIGNALS_AX.yaxis.set_units(units.mV)
                    SHOW_SIGNALS_AX.xaxis.set_units(units.us)
                    SHOW_SIGNALS_AXx.axvline(0, color='k', linestyle='--', label='Trigger')
                    SHOW_SIGNALS_AX.legend()
                    plt.show()

                if(EXIT_FLAG==True):
                    print("Data acquisition is terminating...")
                    return
                
    #reproting of the data aquisition exit
    now = datetime.datetime.now()
    print("")
    print("******************************************")
    print("Data acquisition is terminated on event")
    print(str(now))
    print("******************************************")


In [12]:
colors_list=dc.get_colors(5)
Real_time_FolderTrackerThread= threading.Thread(target=DAQCard,args=(2200,0.1,5000,[0,3],0,1,0.1,False,None,True,50,"None",None,colors_list,None,None))
            
Real_time_FolderTrackerThread.start()   
#global EXIT_FLAG
#EXIT_FLAG=True


******************************************
Data received: 2025-12-12 10:41:32.400217
Data length/chan.: 112
Chans.num.: 8
Frame num.:1
Trig. chan.: 0
Trig.timestamp: 0.01 ms
Trig. val.:2.7246093750000004 V
******************************************


******************************************
Data received: 2025-12-12 10:41:32.406799
Data length/chan.: 112
Chans.num.: 8
Frame num.:2
Trig. chan.: 0
Trig.timestamp: 0.01 ms
Trig. val.:2.655792236328125 V
******************************************


******************************************
Data received: 2025-12-12 10:41:32.411917
Data length/chan.: 112
Chans.num.: 8
Frame num.:3
Trig. chan.: 0
Trig.timestamp: 0.01 ms
Trig. val.:2.4896240234375 V
******************************************


******************************************
Data received: 2025-12-12 10:41:32.416751
Data length/chan.: 112
Chans.num.: 8
Frame num.:4
Trig. chan.: 0
Trig.timestamp: 0.01 ms
Trig. val.:2.0123291015625 V
******************************************


**

In [13]:
Real_time_FolderTrackerThread.do_run = False

In [5]:
global EXIT_FLAG
EXIT_FLAG=True
Real_time_FolderTrackerThread.stop()            

AttributeError: 'Thread' object has no attribute 'stop'

In [5]:
unit_data_V = channel0.convert_data(data_transfer.buffer[channel1, :], units.V)
fig1, ax1 = plt.subplots()
print(channel1)
print("\tMinimum: {:.3~P}".format(np.min(unit_data_V)))
print("\tMaximum: {:.3~P}".format(np.max(unit_data_V)))
ax1.clear()
ax1.plot(time_data_s, unit_data_V, label=f"{channel0}")
ax1.yaxis.set_units(units.mV)
ax1.xaxis.set_units(units.us)
ax1.axvline(0, color='k', linestyle='--', label='Trigger')
ax1.legend()
plt.show()

NameError: name 'channel0' is not defined

In [25]:
vadf=4
if(isinstance(vadf,str)):
    print("True")
if(isinstance(vadf,int)):
    print("False")

False


In [ ]:
"""
Spectrum Instrumentation GmbH (c)

1_acq_single.py

Shows a simple Standard acquisition (recording) mode example using only the few necessary commands
- connect a function generator that generates a sine wave with 10-100 kHz frequency and 200 mV amplitude to channel 0

Example for analog recording cards (digitizers) for the the M2p, M4i, M4x and M5i card-families.

See the README file in the parent folder of this examples directory for information about how to use this example.

See the LICENSE file for the conditions under which this software may be used and distributed.
"""

import spcm
from spcm import units # spcm uses the pint library for unit handling (units is a UnitRegistry object)

import numpy as np
import matplotlib.pyplot as plt


card : spcm.Card

# with spcm.Card('/dev/spcm0') as card:                         # if you want to open a specific card
# with spcm.Card('TCPIP::192.168.1.10::inst0::INSTR') as card:  # if you want to open a remote card
# with spcm.Card(serial_number=12345) as card:                  # if you want to open a card by its serial number
with spcm.Card('/dev/spcm1') as card:            # if you want to open the first card of a specific type
    
    # do a simple standard setup
    card.card_mode(spcm.SPC_REC_STD_SINGLE)     # single trigger standard mode
    card.timeout(5 * units.s)                     # timeout 5 s

    trigger = spcm.Trigger(card)
    trigger.or_mask(spcm.SPC_TMASK_NONE)       # trigger set to none #software
    trigger.and_mask(spcm.SPC_TMASK_NONE)      # no AND mask

    clock = spcm.Clock(card)
    clock.mode(spcm.SPC_CM_INTPLL)            # clock mode internal PLL
    clock.sample_rate(2 * units.MHz, return_unit=units.MHz)
    
    # setup the channels
    channel0, = spcm.Channels(card, card_enable=spcm.CHANNEL0) # enable channel 0
    channel0.amp(10000 * units.mV)
    channel0.offset(0 * units.mV)
    channel0.termination(0)
    # channels.coupling(spcm.COUPLING_DC)

    # Channel triggering
    # trigger.ch_or_mask0(channel0.ch_mask())
    # trigger.ch_mode(channel0, spcm.SPC_TM_POS)
    # trigger.ch_level0(channel0, 0 * units.mV, return_unit=units.mV)
    trigger.and_mask(spcm.SPC_TMASK_NONE)
    trigger.or_mask(spcm.SPC_TMASK_EXT0)
    trigger.ext0_mode(spcm.SPC_TM_POS)
    trigger.ext0_coupling(spcm.COUPLING_DC)
    trigger.termination(termination=0)
    trigger.ext0_level0(
        float(1) * units.V
    )

    # define the data buffer
    data_transfer = spcm.DataTransfer(card)
    data_transfer.duration(10000*units.us, post_trigger_duration=80*units.us)
    # Start DMA transfer
    data_transfer.start_buffer_transfer(spcm.M2CMD_DATA_STARTDMA)
    
    # start card
    print(f"Starting acquisition on card {card}")
    card.start(spcm.M2CMD_CARD_ENABLETRIGGER, spcm.M2CMD_DATA_WAITDMA)

    print("Finished acquiring...")

    # Plot the acquired data
    time_data_s = data_transfer.time_data()
    fig, ax = plt.subplots()
    unit_data_V = channel0.convert_data(data_transfer.buffer[channel0, :], units.V)
    print(channel0)
    print("\tMinimum: {:.3~P}".format(np.min(unit_data_V)))
    print("\tMaximum: {:.3~P}".format(np.max(unit_data_V)))
    ax.plot(time_data_s, unit_data_V, label=f"{channel0}")
    ax.yaxis.set_units(units.mV)
    ax.xaxis.set_units(units.us)
    ax.axvline(0, color='k', linestyle='--', label='Trigger')
    ax.legend()
    plt.show()
